In [ ]:
# Adınız: GAMZE
# Soyadınız: BÜYÜKTORUN
# Okul Numaranız: 2212721033
# GitHub Repo Bağlantısı: https://github.com/GamzeBuyuktorun/Araba_Fiyat_Tahmini
# Proje Konusu: Çoklu Doğrusal Regresyon (MLR) ile Araç Fiyatı Tahmini

In [ ]:
# Gerekli Kütüphaneleri Kurma
!pip install pandas numpy scikit-learn joblib statsmodels

# Google Drive'ı Bağlama
from google.colab import drive
import os
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import statsmodels.api as sm

# ⚠️ DOSYA YOLUNUZU BURAYA GÜNCELLEYİN
file_path = '/content/drive/My Drive/car data.csv' # Veya yüklediğiniz dosyanın adı
TARGET = 'selling_price' # Tahmin edilecek sütun adını doğru belirleyelim

try:
    df = pd.read_csv(file_path)
    print("✅ Veri seti yüklendi.")
except FileNotFoundError:
    print("❌ HATA: Lütfen dosya yolunu ve dosya adını kontrol edin.")
    exit()

# İlk kontrol
print("İlk 5 satır:")
print(df.head())

print(f"\nVeri seti kolonları: {df.columns.tolist()}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Veri seti yüklendi.
İlk 5 satır:
  Car_Name  Year  Selling_Price  Present_Price  Driven_kms Fuel_Type  \
0     ritz  2014           3.35           5.59       27000    Petrol   
1      sx4  2013           4.75           9.54       43000    Diesel   
2     ciaz  2017           7.25           9.85        6900    Petrol   
3  wagon r  2011           2.85           4.15        5200    Petrol   
4    swift  2014           4.60           6.87       42450    Diesel   

  Selling_type Transmission  Owner  
0       Dealer       Manual      0  
1       Dealer       Manual      0  
2       Dealer       Manual      0  
3       Dealer       Manual      0  
4       Dealer       Manual      0  

Veri seti kolonları: ['Car_Name', 'Year', 'Selling_Price', 'Present_Price', 'Driven_kms', 'Fuel_Type', 'Selling_type', 'Transmission', 'Owner']


In [ ]:
## 1. Veri Ön İşleme (Data Preprocessing) Aşaması

# Kayıp Veri (Missing Values) Analizi
print("\n--- Kayıp Veri Kontrolü ---")
print(df.isnull().sum())

# Çoğu zaman bu veri setinde kayıp veri bulunmaz.
# Eğer varsa (örneğin 'Mileage' veya 'Engine' olsaydı), ortalama ile doldurma (fillna) yapılabilirdi.
# Şu anki durumda eksik veri yok, bu yüzden işlem yapmaya gerek yok.


--- Kayıp Veri Kontrolü ---
Car_Name         0
Year             0
Selling_Price    0
Present_Price    0
Driven_kms       0
Fuel_Type        0
Selling_type     0
Transmission     0
Owner            0
dtype: int64


In [ ]:
# Kolon isimlerini basitleştirme ve Yaş hesaplama
df = df.rename(columns={'Selling_Price': 'Price', 'Present_Price': 'Present_Price_Lakh', 'Driven_kms': 'Kms', 'Selling_type': 'Seller_Type'})
df['Age'] = 2025 - df['Year'] # Daha anlamlı bir özellik olduğu için Year yerine Age kullanıyoruz.

# Öznitelik Seçimi: Maksimum 10 olmalıdır. Biz en anlamlı 7 tanesini seçiyoruz.
# 'Car_Name' ve 'Year' atıldı.
FEATURES = ['Present_Price_Lakh', 'Kms', 'Fuel_Type', 'Seller_Type', 'Transmission', 'Owner', 'Age']
TARGET = 'Price'

df = df[FEATURES + [TARGET]].copy()

print(f"\nFinal Kullanılacak Öznitelikler: {df.columns.tolist()}")


Final Kullanılacak Öznitelikler: ['Present_Price_Lakh', 'Kms', 'Fuel_Type', 'Seller_Type', 'Transmission', 'Owner', 'Age', 'Price']


In [ ]:
# Kategorik Veri Kontrolü
categorical_features = ['Fuel_Type', 'Seller_Type', 'Transmission']

# NEDEN: MLR'nin tahmin performansını artırmak ve kategoriler arasında yanlış sıralama varsayımı yapmasını engellemek için.
df_processed = pd.get_dummies(df, columns=categorical_features, drop_first=True, dtype=int)

print("\n✅ Kategorik Veriler One-Hot Encoding ile işlendi. İlk 5 satır:")
print(df_processed.head())


✅ Kategorik Veriler One-Hot Encoding ile işlendi. İlk 5 satır:
   Present_Price_Lakh    Kms  Owner  Age  Price  Fuel_Type_Diesel  \
0                5.59  27000      0   11   3.35                 0   
1                9.54  43000      0   12   4.75                 1   
2                9.85   6900      0    8   7.25                 0   
3                4.15   5200      0   14   2.85                 0   
4                6.87  42450      0   11   4.60                 1   

   Fuel_Type_Petrol  Seller_Type_Individual  Transmission_Manual  
0                 1                       0                    1  
1                 0                       0                    1  
2                 1                       0                    1  
3                 1                       0                    1  
4                 0                       0                    1  


In [ ]:
from sklearn.preprocessing import StandardScaler
import joblib

# Ölçekleme için sayısal verileri seçiyoruz (Kukla değişkenler ölçeklenmez)
numeric_features = ['Present_Price_Lakh', 'Kms', 'Age']

# NEDEN: Farklı ölçeklerdeki özniteliklerin katsayılara olan aşırı etkisini dengelemek için.
scaler = StandardScaler()

# Ölçekleyiciyi eğitiyor ve veriyi dönüştürüyoruz
df_processed[numeric_features] = scaler.fit_transform(df_processed[numeric_features])

# Ölçekleyiciyi kaydediyoruz (Flask'ta kullanılacak)
joblib.dump(scaler, 'scaler.pkl')

print("\n✅ Sayısal Veriler StandardScaler ile ölçeklendi.")
print("Ölçekleyici 'scaler.pkl' olarak kaydedildi.")
print(df_processed.head())


✅ Sayısal Veriler StandardScaler ile ölçeklendi.
Ölçekleyici 'scaler.pkl' olarak kaydedildi.
   Present_Price_Lakh       Kms  Owner       Age  Price  Fuel_Type_Diesel  \
0           -0.236256 -0.256224      0 -0.128897   3.35                 0   
1            0.221544  0.155911      0  0.217514   4.75                 1   
2            0.257472 -0.773969      0 -1.168129   7.25                 0   
3           -0.403151 -0.817758      0  0.910335   2.85                 0   
4           -0.087906  0.141743      0 -0.128897   4.60                 1   

   Fuel_Type_Petrol  Seller_Type_Individual  Transmission_Manual  
0                 1                       0                    1  
1                 0                       0                    1  
2                 1                       0                    1  
3                 1                       0                    1  
4                 0                       0                    1  


In [ ]:
## 2. Geriye Doğru Eleme (Backward Elimination)

import statsmodels.api as sm
import joblib

# Bağımsız değişkenleri (X) ve hedef değişkeni (y) ayırma
TARGET = 'Price'
X = df_processed.drop(TARGET, axis=1)
y = df_processed[TARGET]

# Sabit Terimi (Intercept) Ekleme
# OLS (Ordinary Least Squares) modelinde sabit terim (b0) için gereklidir.
X = sm.add_constant(X)
initial_features = X.columns.tolist()

# Eleme Süreci Ayarları
p_value_threshold = 0.05
max_p_value = 1.0

print("--- Geriye Doğru Eleme Süreci Başladı (p-value > 0.05 olanlar elenir) ---")

# Backward Elimination döngüsü
while max_p_value > p_value_threshold:
    # OLS modelini kurma ve eğitme
    model_ols = sm.OLS(y, X).fit()
    p_values = model_ols.pvalues

    # En yüksek p-value'ya sahip özniteliği bulma (sabit hariç)
    max_p_value = p_values.drop('const', errors='ignore').max()

    if max_p_value > p_value_threshold:
        # Eleme için özniteliği bulma
        feature_to_remove = p_values.drop('const', errors='ignore').idxmax()
        X = X.drop(feature_to_remove, axis=1)
        print(f"Eleme: '{feature_to_remove}' (p-value: {max_p_value:.4f})")
    else:
        print("\nEleme sonlandı. Kalan tüm öznitelikler istatistiksel olarak anlamlıdır (p < 0.05).")
        break

# Final Model Özelliklerini kaydetme (Flask için)
final_model_features = X.columns.tolist()
if 'const' in final_model_features:
    final_model_features.remove('const')

joblib.dump(final_model_features, 'final_features.pkl')

print(f"\nFinal Modelde Kalan Özellikler: {final_model_features}")
print("✅ Final özellik listesi 'final_features.pkl' olarak kaydedildi.")

--- Geriye Doğru Eleme Süreci Başladı (p-value > 0.05 olanlar elenir) ---
Eleme: 'Fuel_Type_Petrol' (p-value: 0.6298)
Eleme: 'Owner' (p-value: 0.1120)

Eleme sonlandı. Kalan tüm öznitelikler istatistiksel olarak anlamlıdır (p < 0.05).

Final Modelde Kalan Özellikler: ['Present_Price_Lakh', 'Kms', 'Age', 'Fuel_Type_Diesel', 'Seller_Type_Individual', 'Transmission_Manual']
✅ Final özellik listesi 'final_features.pkl' olarak kaydedildi.


In [ ]:
## 3. Model Kurulumu ve Değerlendirme

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import numpy as np
import joblib

# Backward Elimination sonucu kalan öznitelikleri seçme
final_model_features = joblib.load('final_features.pkl')

# X ve y değişkenlerini hazırlama
X_final = df_processed[final_model_features] # df_processed, ölçeklenmiş ve encoding yapılmış veri
TARGET = 'Price'
y = df_processed[TARGET]

# Eğitim ve Test Setlerine Ayırma (Verinin %80'i eğitim, %20'si test)
# NEDEN: Modelin, görmediği yeni veriler üzerindeki performansını objektif olarak ölçmek için.
X_train, X_test, y_train, y_test = train_test_split(X_final, y, test_size=0.2, random_state=42)

# Çoklu Doğrusal Regresyon Modelini Kurma ve Eğitme
mlr_model = LinearRegression()
mlr_model.fit(X_train, y_train)

# Tahmin Yapma
y_pred = mlr_model.predict(X_test)

# Modeli Kaydetme (Flask için)
joblib.dump(mlr_model, 'model.pkl')
print("✅ Eğitimli MLR modeli 'model.pkl' olarak kaydedildi.")

# --- Model Değerlendirme Metrikleri ---
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print("\n--- Model Değerlendirme Sonuçları ---")
print(f"R² (R Kare): {r2:.4f}")
print(f"MAE (Ortalama Mutlak Hata): {mae:.4f}")
print(f"MSE (Ortalama Karesel Hata): {mse:.4f}")

# --- Sonuç Yorumu ---
print("\n--- Sonuç Yorumu ---")
print(f"1. R² değeri ({r2:.4f}): Modele dahil ettiğimiz öznitelikler, Fiyat değişkenliğinin yaklaşık %{r2*100:.2f}'ünü açıklamaktadır.")
print(f"2. MAE ({mae:.4f}): Tahmin edilen fiyatlarımız, ortalama olarak gerçek fiyattan {mae:.4f} birim (Lakh) sapmaktadır.")

✅ Eğitimli MLR modeli 'model.pkl' olarak kaydedildi.

--- Model Değerlendirme Sonuçları ---
R² (R Kare): 0.8543
MAE (Ortalama Mutlak Hata): 1.1860
MSE (Ortalama Karesel Hata): 3.3555

--- Sonuç Yorumu ---
1. R² değeri (0.8543): Modele dahil ettiğimiz öznitelikler, Fiyat değişkenliğinin yaklaşık %85.43'ünü açıklamaktadır.
2. MAE (1.1860): Tahmin edilen fiyatlarımız, ortalama olarak gerçek fiyattan 1.1860 birim (Lakh) sapmaktadır.


In [ ]:
## 4. Flask Arayüz Uygulaması İçin Dosya İndirme

from google.colab import files

print("Lütfen çıkan bağlantıları kullanarak 3 dosyayı da indirin:")
files.download('model.pkl')         # Eğitilmiş MLR modeli
files.download('scaler.pkl')        # Sayısal verilerin ölçekleyicisi
files.download('final_features.pkl')# Backward Elimination sonrası kalan özellik listesi

Lütfen çıkan bağlantıları kullanarak 3 dosyayı da indirin:


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>